## Testing Geoapify Places API

In [16]:
import sys
import pandas as pd
# change working directory to the root of the project
sys.path.append('../')
from src.methods.data_ingestion import call_api

In [65]:
request_params = {
    "apiKey": "493bbfea1f124b27906cf052aa330680",
    "categories": "entertainment",
    "limit": "20",
}
filters = {
        "circle": "-1.257677,51.7520131,5000",
}

url = 'https://api.geoapify.com/v2/places?'

# append parameters to the url
for key, value in request_params.items():
    url = url.replace('?', f'?{key}={value}&')

# append filters to the url
for key, value in filters.items():
    url = url.replace('?', f'?filter={key}:{value}&')

# remove the last '&' from the url
url = url[:-1]
url


'https://api.geoapify.com/v2/places?filter=circle:-1.257677,51.7520131,5000&limit=20&categories=entertainment&apiKey=493bbfea1f124b27906cf052aa330680'

Find all museums in a 5000m radium from Oxford city centre

In [66]:
res = call_api(url)
res

{'type': 'FeatureCollection',
 'features': [{'type': 'Feature',
   'properties': {'name': 'Ashmolean Museum',
    'country': 'United Kingdom',
    'country_code': 'gb',
    'state': 'England',
    'county': 'Oxfordshire',
    'city': 'Oxford',
    'postcode': 'OX1 2PH',
    'district': "St Thomas'",
    'suburb': 'City Centre',
    'street': 'Beaumont Street',
    'lon': -1.2602574960821362,
    'lat': 51.7554961,
    'state_code': 'ENG',
    'formatted': 'Ashmolean Museum, Beaumont Street, Oxford, OX1 2PH, United Kingdom',
    'address_line1': 'Ashmolean Museum',
    'address_line2': 'Beaumont Street, Oxford, OX1 2PH, United Kingdom',
    'categories': ['building',
     'building.tourism',
     'entertainment',
     'entertainment.museum',
     'no_fee',
     'no_fee.no'],
    'details': ['details',
     'details.building',
     'details.contact',
     'details.facilities',
     'details.wiki_and_media'],
    'datasource': {'sourcename': 'openstreetmap',
     'attribution': '© OpenStr

In [68]:
len(res['features'])

20

In [70]:
df = pd.DataFrame.from_dict(res['features'][13]['properties'], orient='index').T
# expand datasource column into multiple columns
df = pd.concat([df.drop(['datasource','place_id'], axis=1), df['datasource'].apply(pd.Series)], axis=1)
# expand raw column into multiple columns
df = pd.concat([df.drop(['raw'], axis=1), df['raw'].apply(pd.Series)], axis=1)
df.T

,0
name,Pegasus Theatre
country,United Kingdom
country_code,gb
state,England
county,Oxfordshire
city,Oxford
postcode,OX4 1RE
district,Robin Hood
street,Magdalen Road
lon,-1.23637


In [83]:
category_dict = {}
for i in range(0,len(res['features'])):
    name = res['features'][i]['properties']['name']
    categories = res['features'][i]['properties']['categories']
    category_dict[name] = categories

entertainment_df = pd.DataFrame.from_dict(category_dict, orient='index')
entertainment_df

,0,1,2,3,4,5,6,7,8
Ashmolean Museum,building,building.tourism,entertainment,entertainment.museum,no_fee,no_fee.no,None,None,None
Modern Art Oxford,entertainment,entertainment.culture,entertainment.culture.arts_centre,wheelchair,wheelchair.yes,None,None,None,None
Sheldonian Theatre,building,building.entertainment,building.tourism,building.university,entertainment,entertainment.culture,entertainment.culture.theatre,tourism,tourism.attraction
Museum of the History of Science,building,building.tourism,entertainment,entertainment.museum,no_fee,no_fee.no,wheelchair,wheelchair.limited,None
Oxford University Museum of Natural History,building,building.tourism,entertainment,entertainment.museum,wheelchair,wheelchair.yes,None,None,None
Burton Taylor Studio,building,building.entertainment,entertainment,entertainment.culture,entertainment.culture.theatre,None,None,None,None
The Ultimate Picture Palace,building,building.entertainment,entertainment,entertainment.cinema,None,None,None,None,None
Museum of Oxford,entertainment,entertainment.museum,None,None,None,None,None,None,None
Oxford Playhouse,building,building.entertainment,entertainment,entertainment.culture,entertainment.culture.theatre,None,None,None,None
Bate Collection of Musical Instruments,entertainment,entertainment.museum,no_fee,no_fee.no,None,None,None,None,None


In [ ]:
entertainment_df.to_csv('entertainment.csv', index=False)